In [1]:
import torch
import h5py

import numpy as np
import os

import torch
from torch import nn
from torch import optim

from PIL import Image
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split

from torchvision import models

import gc

/home/harkhymadhe/miniforge3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/harkhymadhe/miniforge3/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
# Set device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# File paths
electron_file = "dataset/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5"
photon_file = "dataset/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5"

In [3]:
# Load data files
electron_data = h5py.File(name = electron_file)
photon_data = h5py.File(name = photon_file)

In [4]:
electron_data.keys()

<KeysViewHDF5 ['X', 'y']>

In [5]:
# Feature shape
electron_data['X'].shape

(249000, 32, 32, 2)

In [6]:
# Target shape
electron_data['y'].shape

(249000,)

In [7]:
# Feature shape
photon_data['X'].shape

(249000, 32, 32, 2)

In [8]:
# Target shape
photon_data['y'].shape

(249000,)

In [9]:
# Target shape
photon_data['y'].shape

(249000,)

In [10]:
def h5_to_numpy(h5_file):
    X, y = h5_file["X"], h5_file["y"]
    return np.array(X), np.array(y)

In [11]:
electron_data = h5_to_numpy(electron_data)
photon_data = h5_to_numpy(photon_data)

In [12]:
data = np.concatenate([electron_data[0], photon_data[0]], axis = 0)
targets = np.concatenate([electron_data[1].reshape(-1, 1), photon_data[1].reshape(-1, 1)], axis = 0)

In [13]:
del electron_data
del photon_data

In [14]:
gc.collect()

0

In [ ]:
indices = np.array(list(range(data.shape[0])))

In [ ]:
# Shuffle data
data = data[indices]
targets = targets[indices]

In [15]:
data.dtype

dtype('float32')

In [16]:
targets.dtype

dtype('float32')

In [17]:
data[:10].shape

(10, 32, 32, 2)

In [18]:
set(targets.flatten().tolist())

{0.0, 1.0}

In [19]:
dataset = TensorDataset(torch.tensor(data).permute(0, 3, 1, 2), torch.tensor(targets, dtype = torch.int64))

In [20]:
train_ds, test_ds = random_split(dataset, lengths = [.8, .2])

In [21]:
train_dl = DataLoader(train_ds, batch_size=8, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=8, shuffle=True)

In [22]:
gc.collect()

0

In [23]:
model = models.resnet18(pretrained=True)

/home/harkhymadhe/miniforge3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/harkhymadhe/miniforge3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [25]:
for param in model.parameters():
    param.requires_grad_(False)

In [26]:
model.conv1 = nn.Conv2d(2, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [27]:
def initialize_weights(model):
    for (name, weights) in filter(lambda x: x[1].requires_grad, model.named_parameters()):
        try:
            nn.init.kaiming_normal_(weights)
        except:
            nn.init.normal_(weights, 0., 0.05)
    return model

def get_l2_loss(model):
    return sum([x ** 2 for x in model.parameters()])

In [28]:
fc = nn.Sequential(
    nn.Linear(in_features=model.fc.in_features, out_features=model.fc.in_features // 2),
    nn.ReLU(inplace = True),
    nn.Linear(in_features=model.fc.in_features//2, out_features=2),
)

model.fc = fc

In [29]:
model.to("cuda")

model = initialize_weights(model)

In [30]:
criterion = nn.CrossEntropyLoss().cuda()

In [31]:
opt = optim.Adam(
    params = [{
        "params" : model.fc.parameters(),
        "lr": 1e-4
    }],
    lr=1e-6
)

In [32]:
def get_l2_loss(model):
    l2_loss = torch.tensor(0.).cuda()
    l2_loss += sum(map(lambda x: x.pow(2).sum(), filter(lambda x: x.requires_grad, model.parameters())))
    return l2_loss

In [33]:
def collate_function_dl(batch):

    #xs = batch[0].clone()
    #ys = batch[1].clone()

    xs = [item[0].unsqueeze(0) for item in batch]
    ys = [item[1] for item in batch]
    
    xs = torch.cat(xs, dim=0)

    y = torch.tensor(ys).view(-1, 1)
    
    Xs = [torch.rot90(xs, k = _, dims = [-2, -1]) for _ in range(4)]

    return torch.cat(Xs, dim = 0), torch.cat([y for _ in range(4)], dim = 0).view(-1)

def collate_function(batch):

    xs = batch[0].clone()
    ys = batch[1].clone().view(-1, 1)
    
    Xs = [torch.rot90(xs, k = _, dims = [-2, -1]) for _ in range(4)]

    return torch.cat(Xs, dim = 0), torch.cat([ys for _ in range(4)], dim = 0).view(-1)

In [ ]:
EPOCHS = 20
l2_lambda = 0.0004

for epoch in range(EPOCHS):
    for batch in train_dl:
        X, y = collate_function(batch)
        with torch.cuda.amp.autocast():
            pred = model(X.cuda())
        
        pred = torch.log_softmax(pred, dim=-1)
        loss = criterion(pred, y.cuda()) + (l2_lambda * get_l2_loss(model))
        loss.backward()
        opt.step()

        opt.zero_grad()

        print(f"Epoch  {epoch+1}; Loss {round(loss.item(), 4)}")

/home/harkhymadhe/miniforge3/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1706726118919/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch  1; Loss 1.6613
Epoch  1; Loss 1.0548
Epoch  1; Loss 1.3106
Epoch  1; Loss 1.2855
Epoch  1; Loss 1.7171
Epoch  1; Loss 1.2344
Epoch  1; Loss 1.4376
Epoch  1; Loss 0.9949
Epoch  1; Loss 1.3646
Epoch  1; Loss 1.313
Epoch  1; Loss 1.3558
Epoch  1; Loss 1.4537
Epoch  1; Loss 1.3943
Epoch  1; Loss 1.0212
Epoch  1; Loss 1.2995
Epoch  1; Loss 1.3491
Epoch  1; Loss 1.1549
Epoch  1; Loss 1.4408
Epoch  1; Loss 1.5691
Epoch  1; Loss 1.6977
Epoch  1; Loss 1.3656
Epoch  1; Loss 1.0695
Epoch  1; Loss 1.4691
Epoch  1; Loss 1.1748
Epoch  1; Loss 1.1197
Epoch  1; Loss 1.6106
Epoch  1; Loss 1.2785
Epoch  1; Loss 1.3047
Epoch  1; Loss 1.451
Epoch  1; Loss 1.1193
Epoch  1; Loss 1.2585
Epoch  1; Loss 0.8004
Epoch  1; Loss 1.5907
Epoch  1; Loss 1.2863
Epoch  1; Loss 1.2677
Epoch  1; Loss 1.2621
Epoch  1; Loss 1.4312
Epoch  1; Loss 1.3626
Epoch  1; Loss 1.1219
Epoch  1; Loss 1.2973
Epoch  1; Loss 1.7191
Epoch  1; Loss 1.4441
Epoch  1; Loss 1.1898
Epoch  1; Loss 1.3507
Epoch  1; Loss 1.0801
Epoch  1; Lo